In [1]:
import pandas as pd
import numpy as np
import glob
import re

In [34]:
# gather all radiomic data from primary and recurred tumors
rootdir = '/Users/shuang/Documents/Proj_Radiomics/Data/her2'

# get radiomics of all the primary tumor data
fname1 = '{}/her2_Analysis/PETMRI/PETbinwidth0.1_MRItp2_binwidth5/data_all.csv'.format(rootdir)
df_prim_all = pd.read_csv(fname1)

# print(df_prim_all.columns.tolist())

# find all PET radiomics
pat = re.compile('_pet')
feat_names = [ss for ss in df_prim_all.columns.tolist() if re.search('([\w.]+)_pet',ss)]
new_feat_names = [re.search('([\w.]+)_pet', ss).group(1) for ss in df_prim_all.columns.tolist() if re.search('([\w.]+)_pet', ss)]
newer_feat_names = [re.search('([\w.]+)_avg', ss).group(1) if re.search('([\w.]+)_avg', ss) else ss for ss in new_feat_names]

the_col_names = feat_names + ['ptid_side']
df_prim = df_prim_all.loc[:,the_col_names]

# change feature name
col_dict = dict(zip(feat_names, newer_feat_names))
df_prim.rename(col_dict, axis='columns',inplace=True)
df_prim['tumor_type'] = 'Primary'
# print(df_prim.columns.tolist())


json_dir = '{}/her2_ImageFeatures/IsoVoxelSize'.format(rootdir)
all_jsons = glob.glob('{}/*.json'.format(json_dir))

df_recur = pd.DataFrame()
for jj in all_jsons:
    df_tmp = pd.read_json(jj)
    df_recur = df_recur.append(df_tmp, ignore_index=True)
df_recur['FOstats_min'] = df_recur['FOstats_minmax'].apply(lambda x: x[0])
df_recur['FOstats_max'] = df_recur['FOstats_minmax'].apply(lambda x: x[1])
df_recur.drop(columns=['FOstats_minmax'],inplace=True)

# get the average of texture features




    
df_recur['tumor_type'] = df_recur['tumor_tag'].map(lambda x: '_'.join(['Recur',x]))
df_recur['ptid_side'] = df_recur[['pt_id','breast_side']].apply(lambda x: '{}_{}'.format(x[0], x[1]), axis=1)

col_of_interest = df_prim.columns.tolist()

df_recur_oi = df_recur.loc[:,col_of_interest]
df_prim_oi = df_prim.loc[:,col_of_interest]

# combine primary and recur tumor DFs
df_all = pd.concat([df_prim_oi, df_recur_oi], ignore_index=True)



   FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0     8163.651367         5.458992          2.368069      4.578343   

                 FOstats_minmax  FOstats_skewness  FOstats_uniformity  \
0  [1.6559542418, 7.5018849373]         -0.106327            0.025753   

   FOstats_variance  ShapeSize_compactness1  ShapeSize_compactness2  \
0          1.404941                0.262913                0.249885   

                         ...                          \
0                        ...                           

                                        texture_imc1  \
0  [-0.4603321519, -0.4525922248, -0.4614210574, ...   

                                        texture_imc2  \
0  [0.9966337826, 0.9964336843, 0.9967487927, 0.9...   

                                     texture_inv_var  \
0  [0.1194943884, 0.131475588, 0.0888064914, 0.14...   

                                     texture_maxprob  \
0  [0.0092592593, 0.012145749, 0.0093023256, 0.00...   

    

   FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0     8163.651367         5.458992          2.368069      4.578343   
1    21483.960938         4.953702          2.829250      6.438882   
2     6372.353516         4.575003          6.384124      1.400479   
3      871.530823         3.483647          3.179788      1.027038   
4   200519.062500         6.291970          2.295061      3.946515   
5     4564.866699         5.075825          3.117624      2.541602   

                  FOstats_minmax  FOstats_skewness  FOstats_uniformity  \
0   [1.6559542418, 7.5018849373]         -0.106327            0.025753   
1   [3.9444715977, 8.7964086533]          0.157781            0.038055   
2   [0.4701008797, 6.1134667397]          1.887966            0.063987   
3   [0.5138121247, 2.0159211159]          0.935619            0.111005   
4  [0.1897505522, 10.3748750687]          0.579612            0.014933   
5   [1.0293451548, 5.5833616257]          0.666548            0.0

   FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0     8163.651367         5.458992          2.368069      4.578343   
1    21483.960938         4.953702          2.829250      6.438882   
2     6372.353516         4.575003          6.384124      1.400479   
3      871.530823         3.483647          3.179788      1.027038   
4   200519.062500         6.291970          2.295061      3.946515   
5     4564.866699         5.075825          3.117624      2.541602   
6     8606.171875         5.260624          2.941920      2.652999   
7    10375.710938         5.468245          4.586837      2.890363   

                  FOstats_minmax  FOstats_skewness  FOstats_uniformity  \
0   [1.6559542418, 7.5018849373]         -0.106327            0.025753   
1   [3.9444715977, 8.7964086533]          0.157781            0.038055   
2   [0.4701008797, 6.1134667397]          1.887966            0.063987   
3   [0.5138121247, 2.0159211159]          0.935619            0.111005   

   FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0     8163.651367         5.458992          2.368069      4.578343   
1    21483.960938         4.953702          2.829250      6.438882   
2     6372.353516         4.575003          6.384124      1.400479   
3      871.530823         3.483647          3.179788      1.027038   
4   200519.062500         6.291970          2.295061      3.946515   
5     4564.866699         5.075825          3.117624      2.541602   
6     8606.171875         5.260624          2.941920      2.652999   
7    10375.710938         5.468245          4.586837      2.890363   
8     6506.383789         4.886855          2.778357      2.696445   
9    58554.046875         5.788815          2.274279      3.608573   

                  FOstats_minmax  FOstats_skewness  FOstats_uniformity  \
0   [1.6559542418, 7.5018849373]         -0.106327            0.025753   
1   [3.9444715977, 8.7964086533]          0.157781            0.038055   
2   [0.

    FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0      8163.651367         5.458992          2.368069      4.578343   
1     21483.960938         4.953702          2.829250      6.438882   
2      6372.353516         4.575003          6.384124      1.400479   
3       871.530823         3.483647          3.179788      1.027038   
4    200519.062500         6.291970          2.295061      3.946515   
5      4564.866699         5.075825          3.117624      2.541602   
6      8606.171875         5.260624          2.941920      2.652999   
7     10375.710938         5.468245          4.586837      2.890363   
8      6506.383789         4.886855          2.778357      2.696445   
9     58554.046875         5.788815          2.274279      3.608573   
10   237548.734375         6.187117          3.056798      3.817431   
11    43619.144531         6.392015          3.862082      4.426337   

                   FOstats_minmax  FOstats_skewness  FOstats_uniformity  \
0

...

    FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0      8163.651367         5.458992          2.368069      4.578343   
1     21483.960938         4.953702          2.829250      6.438882   
2      6372.353516         4.575003          6.384124      1.400479   
3       871.530823         3.483647          3.179788      1.027038   
4    200519.062500         6.291970          2.295061      3.946515   
5      4564.866699         5.075825          3.117624      2.541602   
6      8606.171875         5.260624          2.941920      2.652999   
7     10375.710938         5.468245          4.586837      2.890363   
8      6506.383789         4.886855          2.778357      2.696445   
9     58554.046875         5.788815          2.274279      3.608573   
10   237548.734375         6.187117          3.056798      3.817431   
11    43619.144531         6.392015          3.862082      4.426337   
12    43738.085938         6.038567          2.671116      4.186522   
13   1

    FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0      8163.651367         5.458992          2.368069      4.578343   
1     21483.960938         4.953702          2.829250      6.438882   
2      6372.353516         4.575003          6.384124      1.400479   
3       871.530823         3.483647          3.179788      1.027038   
4    200519.062500         6.291970          2.295061      3.946515   
5      4564.866699         5.075825          3.117624      2.541602   
6      8606.171875         5.260624          2.941920      2.652999   
7     10375.710938         5.468245          4.586837      2.890363   
8      6506.383789         4.886855          2.778357      2.696445   
9     58554.046875         5.788815          2.274279      3.608573   
10   237548.734375         6.187117          3.056798      3.817431   
11    43619.144531         6.392015          3.862082      4.426337   
12    43738.085938         6.038567          2.671116      4.186522   
13   1

    FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0      8163.651367         5.458992          2.368069      4.578343   
1     21483.960938         4.953702          2.829250      6.438882   
2      6372.353516         4.575003          6.384124      1.400479   
3       871.530823         3.483647          3.179788      1.027038   
4    200519.062500         6.291970          2.295061      3.946515   
5      4564.866699         5.075825          3.117624      2.541602   
6      8606.171875         5.260624          2.941920      2.652999   
7     10375.710938         5.468245          4.586837      2.890363   
8      6506.383789         4.886855          2.778357      2.696445   
9     58554.046875         5.788815          2.274279      3.608573   
10   237548.734375         6.187117          3.056798      3.817431   
11    43619.144531         6.392015          3.862082      4.426337   
12    43738.085938         6.038567          2.671116      4.186522   
13   1

    FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0      8163.651367         5.458992          2.368069      4.578343   
1     21483.960938         4.953702          2.829250      6.438882   
2      6372.353516         4.575003          6.384124      1.400479   
3       871.530823         3.483647          3.179788      1.027038   
4    200519.062500         6.291970          2.295061      3.946515   
5      4564.866699         5.075825          3.117624      2.541602   
6      8606.171875         5.260624          2.941920      2.652999   
7     10375.710938         5.468245          4.586837      2.890363   
8      6506.383789         4.886855          2.778357      2.696445   
9     58554.046875         5.788815          2.274279      3.608573   
10   237548.734375         6.187117          3.056798      3.817431   
11    43619.144531         6.392015          3.862082      4.426337   
12    43738.085938         6.038567          2.671116      4.186522   
13   1

    FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
0      8163.651367         5.458992          2.368069      4.578343   
1     21483.960938         4.953702          2.829250      6.438882   
2      6372.353516         4.575003          6.384124      1.400479   
3       871.530823         3.483647          3.179788      1.027038   
4    200519.062500         6.291970          2.295061      3.946515   
5      4564.866699         5.075825          3.117624      2.541602   
6      8606.171875         5.260624          2.941920      2.652999   
7     10375.710938         5.468245          4.586837      2.890363   
8      6506.383789         4.886855          2.778357      2.696445   
9     58554.046875         5.788815          2.274279      3.608573   
10   237548.734375         6.187117          3.056798      3.817431   
11    43619.144531         6.392015          3.862082      4.426337   
12    43738.085938         6.038567          2.671116      4.186522   
13   1

#### to do list
- get all the recur tumor radiomics into on data frame by pt_id, tumor_tag, and tumor_type as 'recur_xxxx' where xxx is the site of recurrence?
- get the appropriate data from data_all.csv and mark the tumor_type as 'prim' based on the pt_id from the recur tumor list (only take the PET radiomics)
- combine the two data frame together via concatenate or etc. (this is NOT a JOIN or MERGE operation)
- use this as the input data for the bokeh circle plot to compare the radiomics across all tumors for each radiomic feature category

In [33]:
# the data ready for bokeh plot
ptid_sides = df_recur_oi.ptid_side.tolist()

ptidsd_oi = ptid_sides[0]
df1 = df_all[df_all['ptid_side'] == ptidsd_oi]
print(df1)
val_vars = set(df1.columns.tolist()).symmetric_difference(['ptid_side','tumor_type'])

# make an appropriate table
df2 = pd.melt(the_df, id_vars=['ptid_side','tumor_type'], value_vars=val_vars)
print(df2)
# print(df2.columns.tolist())
df3 = df2.pivot(index='variable',columns='tumor_type',values='value')
print(df3)


     FOstats_energy  FOstats_entropy  FOstats_kurtosis  FOstats_mean  \
93     39630.937500         3.122532          2.681443      2.578717   
118     8163.651367         5.458992          2.368069      4.578343   
119    21483.960938         4.953702          2.829250      6.438882   

     FOstats_min  FOstats_max  FOstats_skewness  FOstats_uniformity  \
93      2.252231     3.294024          0.653858            0.127763   
118     1.655954     7.501885         -0.106327            0.025753   
119     3.944472     8.796409          0.157781            0.038055   

     FOstats_variance  ShapeSize_compactness1      ...       \
93           0.052561                2.245520      ...        
118          1.404941                0.262913      ...        
119          0.666209                0.316147      ...        

                                   texture_sum_entropy  \
93                                             3.94457   
118  [6.0773582804, 6.1895524304, 6.1065401806, 6.2...   

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.plotting import figure, show